In [52]:
# -*- coding: utf-8 -*-
"""
Yelp API v2.0 code sample.
This program demonstrates the capability of the Yelp API version 2.0
by using the Search API to query for businesses by a search term and location,
and the Business API to query additional information about the top result
from the search query.
Please refer to http://www.yelp.com/developers/documentation for the API documentation.
This program requires the Python oauth2 library, which you can install via:
`pip install -r requirements.txt`.
Sample usage of the program:
`python sample.py --term="bars" --location="San Francisco, CA"`
This program only works with python 2.7, not 3
"""

#import argparse
import json
import pprint
# import sys
import urllib
import urllib2
import oauth2
from APIkeys import keys
# from bs4 import BeautifulSoup
# import lxml
import pymysql
from KAsql import query_SQL
import pandas as pd
import numpy as np
import string



# Global Variables
API_HOST = 'api.yelp.com'
# DEFAULT_TERM = 'orthopedic surgeon'
# DEFAULT_LOCATION = 'San Francisco, CA'
SEARCH_LIMIT = 20
SEARCH_PATH = '/v2/search/'
BUSINESS_PATH = '/v2/business/'

# OAuth credential placeholders that must be filled in by users.
CONSUMER_KEY = keys.CONSUMER_KEY
CONSUMER_SECRET = keys.CONSUMER_SECRET
TOKEN = keys.TOKEN
TOKEN_SECRET = keys.TOKEN_SECRET


In [53]:
def request(host, path, url_params=None):
    """Prepares OAuth authentication and sends the request to the API.
    Args:
        host (str): The domain host of the API.
        path (str): The path of the API after the domain.
        url_params (dict): An optional set of query parameters in the request.
    Returns:
        dict: The JSON response from the request.
    Raises:
        urllib2.HTTPError: An error occurs from the HTTP request.
    """
    url_params = url_params or {}
    url = 'http://{0}{1}?'.format(host, urllib.quote(path.encode('utf8')))

    consumer = oauth2.Consumer(CONSUMER_KEY, CONSUMER_SECRET)
    oauth_request = oauth2.Request(method="GET", url=url, parameters=url_params)

    oauth_request.update(
        {
            'oauth_nonce': oauth2.generate_nonce(),
            'oauth_timestamp': oauth2.generate_timestamp(),
            'oauth_token': TOKEN,
            'oauth_consumer_key': CONSUMER_KEY
        }
    )
    token = oauth2.Token(TOKEN, TOKEN_SECRET)
    oauth_request.sign_request(oauth2.SignatureMethod_HMAC_SHA1(), consumer, token)
    signed_url = oauth_request.to_url()
    
    print (u'Querying {0} ...'.format(url))

    conn = urllib2.urlopen(signed_url, None)
    try:
        response = json.loads(conn.read())
    finally:
        conn.close()

    return response

In [54]:
def search(term, location, limit, offset):
    """Query the Search API by a search term and location.
    Args:
        term (str): The search term passed to the API.
        location (str): The search location passed to the API.
    Returns:
        dict: The JSON response from the request.
    """
    
    url_params = {
        'term': term.replace(' ', '+'),
        'location': location.replace(' ', '+'),
        'limit': SEARCH_LIMIT,
        'offset': offset
    }
    return request(API_HOST, SEARCH_PATH, url_params=url_params)

In [55]:
def query_api(term, location, limit, offset):
    """Queries the API by the input values from the user.
    Args:
        term (str): The search term to query.
        location (str): The location of the business to query.
    """
    response = search(term, location, limit, offset)
#     pprint.pprint(response, indent=2)
    return response

In [19]:
# def get_business(business_id):
#     """Query the Business API by a business ID.
#     Args:
#         business_id (str): The ID of the business to query.
#     Returns:
#         dict: The JSON response from the request.
#     """
#     business_path = BUSINESS_PATH + business_id

#     return request(API_HOST, business_path)

In [20]:
def get_business_from_API(response, term, location):
    businesses = response.get('businesses')

    if not businesses:
        print u'No businesses for {0} in {1} found.'.format(term, location)
        return
    return businesses

In [103]:
def insert_business_SQL(yelp_id, stars, nreviews, categories, term_id, city_id):
    con = False
    try:
        con = pymysql.connect(host='localhost', port=3307, user='root', passwd='', db='ortho')
        with con:
            cur = con.cursor()
            database = 'USE ortho;'
            cur.execute(database)
            #print(type(yelp_id), type(stars), type(reviews)
            #'1' at the end is for the city id, which for now is just SF
            #myvalues = "'" + yelp_id + "'," + str(stars) + "," + str(reviews) + "," + "1"
            myvalues = "'"+yelp_id+"',"+str(stars)+","+str(nreviews)+",'"+str(categories)+"',"+str(term_id)+","+str(city_id)
            print(myvalues)
            sql = 'INSERT INTO business(yelp_id,stars,nreviews,categories, term_id,city_id) VALUES('+myvalues+')'
            print(sql)
            cur.execute(sql)
            #print sql
    except pymysql.Error, e:
        print "Error %d: %s" % (e.args[0],e.args[1])
        sys.exit(1)
    finally:
        if con:
            con.close()
        

In [104]:
def format_business(business):
        yelp_business_id = business[:]['id']
        bus_rating = business[:]['rating']
        bus_nreviews = business[:]['review_count']#for each business
        bus_categories = business[:]['categories']#e.g., Family Practice

        print u'Result for business "{0}" found:'.format(yelp_business_id)

In [106]:
def main():
#     #Useful if running from command line
#     parser = argparse.ArgumentParser()
#     parser.add_argument('-q', '--term', dest='term', default=DEFAULT_TERM, type=str, help='Search term (default: %(default)s)')
#     parser.add_argument('-l', '--location', dest='location', default=DEFAULT_LOCATION, type=str, help='Search location (default: %(default)s)')
#     input_values = parser.parse_args()#This throws an error in Jupyter because there is no input to parse

    #API Search Parameters
    #myterm = 'surgeon'
    myterm = 'orthopedists'
    mylocation = 'San Francisco, CA'
    term_id = 1#'orthopedists'
    city_id = 1#'San Francisco, CA'
    
    # Since the API only queries 20 businesses at a time, loop over offset to get more
    num_bus_toQuery = 300;
    # For debugging, do only one pass
    #num_bus_toQuery = 20;
    for offset in range(0,num_bus_toQuery,20):
        print 'Offset = ',offset
        
        #Query Yelp API
        try:
            response = query_api(term=myterm, location=mylocation, limit=20, offset=0)
        except urllib2.HTTPError as error:
            sys.exit('Encountered HTTP error {0}. Abort program.'.format(error.code))

        businesses = get_business_from_API(response, myterm, mylocation)
        total_nbusinesses = response.get('total')#number of businesses
        print 'Total Businesses found: ',total_nbusinesses
        type(businesses)

        #for i in range(0,2):# for debugging: loop over 3 businesses for each YELP API query
        for i in range(0,20):# loop over 20 businesses (search limit) on each YELP API query
            yelp_business_id = businesses[i]['id']
            b_stars = businesses[i]['rating']#Stars for each business (diff from stars for each review)
            b_nreviews = businesses[i]['review_count']#for each business
            b_categories = businesses[i]['categories']#e.g., Family Practice

            #print u'Result for business "{0}" found:'.format(yelp_business_id)
            
            # Turn categories into a string, and do not push to mysql if it is a chiropractor
            cat_string = ''
            for category in b_categories:
                cat_string = cat_string+category[0].lower()+','
            cat_string = cat_string[0:-1].replace(',',' ')#removes final ,
            
            print(cat_string)
            if cat_string.find('chiro')==-1: 
                #Only push to MySQL if not a chiropractor
                #Add business info to MySQL table: business (other tables are city and review)
                insert_business_SQL(yelp_business_id, b_stars, b_nreviews, cat_string, term_id, city_id)
    
if __name__ == '__main__':
   main()

Offset =  0
Querying http://api.yelp.com/v2/search/? ...
Total Businesses found:  260
sports medicine orthopedists
'james-l-chen-md-san-francisco',5.0,40,'sports medicine orthopedists',1,1
INSERT INTO business(yelp_id,stars,nreviews,categories, term_id,city_id) VALUES('james-l-chen-md-san-francisco',5.0,40,'sports medicine orthopedists',1,1)
sports medicine orthopedists
'elly-s-laroque-md-san-francisco',5.0,34,'sports medicine orthopedists',1,1
INSERT INTO business(yelp_id,stars,nreviews,categories, term_id,city_id) VALUES('elly-s-laroque-md-san-francisco',5.0,34,'sports medicine orthopedists',1,1)
sports medicine orthopedists
'nicholas-colyvas-md-san-francisco',5.0,17,'sports medicine orthopedists',1,1
INSERT INTO business(yelp_id,stars,nreviews,categories, term_id,city_id) VALUES('nicholas-colyvas-md-san-francisco',5.0,17,'sports medicine orthopedists',1,1)
sports medicine orthopedists
'c-benjamin-ma-md-san-francisco',4.0,31,'sports medicine orthopedists',1,1
INSERT INTO business(yel

In [11]:
# use sqlalchemy & pysql
# cmd = 'mysql+pymysql://root@localhost:3307/yelpdata'
# engine = create_engine(cmd) #imported from alchemy
# df.to_sql('review',engine, if_exists='append', index=False)#ignore df index